In [ ]:
Transformers:
    Visualization of BERT model attention weights

In [3]:
from transformers import AutoTokenizer, AutoConfig
from bertviz.transformers_neuron_view import BertModel
from bertviz.neuron_view import show

from torch import nn
import torch
from math import sqrt
import torch.nn.functional as F

In [2]:
model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BertModel.from_pretrained(model_ckpt)
text = "Apple is a good fruit"
show(model,"bert",tokenizer,text,display_mode="light",layer=0,head=8)

100%|████████████████████████████████████████████████████████████████| 440473133/440473133 [03:52<00:00, 1893044.89B/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
text = "Apple is a good fruit"
input = tokenizer(text,return_tensors='pt',add_special_tokens=False)
input.input_ids

tensor([[6207, 2003, 1037, 2204, 5909]])

In [17]:
config = AutoConfig.from_pretrained(model_ckpt)
#config
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
input_emb = token_emb(input.input_ids)
input_emb.size()

torch.Size([1, 5, 768])

In [35]:
query = key = value = input_emb
def scaled_dot_pdt_attn(query , key , value):
    dim_k = key.size(-1)
    scores = torch.bmm(query,key.transpose(1,2))/sqrt(dim_k)
    wts = F.softmax(scores,dim=-1)
    return torch.bmm(wts,value)

In [84]:
class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)

    def forward(self,hidden_state):
        attn_out = scaled_dot_pdt_attn(self.q(hidden_state),self.k(hidden_state),self.v(hidden_state))
        #print(attn_out.size())
        return attn_out

In [85]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        embed_dim = config.hidden_size
        num_heads = config.num_attention_heads
        head_dim = embed_dim // num_heads
        
        self.heads = nn.ModuleList([AttentionHead(embed_dim,head_dim) for _ in range(num_heads)])
        self.out_linear = nn.Linear(embed_dim, embed_dim)
       

    def forward(self,hidden_state):
        x = torch.concat([h(hidden_state) for h in self.heads],dim=-1)
        x = self.out_linear(x)
        return x

In [86]:
multihead_attn = MultiHeadAttention(config)
attn_out = multihead_attn(input_emb)
attn_out.size()

torch.Size([1, 5, 768])

In [87]:
class FF(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.l1 = nn.Linear(config.hidden_size,config.intermediate_size)
        self.l2 = nn.Linear(config.intermediate_size,config.hidden_size)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        
    def forward(self,x):
        x = self.l1(x)
        #print(x.size())
        x = self.gelu(x)
        #print(x.size())
        x = self.l2(x)
        #print(x.size())
        x = self.dropout(x)
        #print(x.size())
        return x

In [88]:
ff = FF(config)
ff_out = ff(attn_out)
ff_out.size()


torch.Size([1, 5, 768])

In [89]:
class Encoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.norm1 = nn.LayerNorm(config.hidden_size)
        self.norm2 = nn.LayerNorm(config.hidden_size)
        self.attention = MultiHeadAttention(config)
        self.ff = FF(config)
        
    def forward(self,x):
        hidden_state = self.norm1(x)
        x = x + self.attention(hidden_state)
        x = x + self.ff(self.norm2(x))
        return x

In [90]:
encoder_layer = Encoder(config)
encoder_layer(input_emb).size()

torch.Size([1, 5, 768])

In [109]:
#Positional Embedding
class Embeddings(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.token_embeddings = nn.Embedding(config.vocab_size,config.hidden_size)
        self.pos_embeddings = nn.Embedding(config.max_position_embeddings,config.hidden_size)
        self.layer_norm = nn.LayerNorm(config.hidden_size,eps = 1e-12)
        self.dropout = nn.Dropout()
    
    def forward(self,input_ids):
        seq_len = input_ids.size(1)
        pos_id = torch.arange(seq_len,dtype = torch.long).unsqueeze(0)
        token_emb = self.token_embeddings(input_ids)
        pos_emb = self.pos_embeddings(pos_id)
        emb = token_emb + pos_emb
        emb = self.layer_norm(emb)
        emb = self.dropout(emb)
        return emb

In [110]:
emb_layer = Embeddings(config)
emb_layer(input.input_ids).size()

torch.Size([1, 5, 768])

In [122]:
class Trans_Encoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.emb = Embeddings(config)
        self.layers = nn.ModuleList([Encoder(config) for _ in range(config.num_hidden_layers)])
    def forward(self,x):
        x = self.emb(x)
        for l in self.layers:
            #print(l)
            x = l(x)
            #print(x.size())
        return x
            
        

In [123]:
enc = Trans_Encoder(config)
enc(input.input_ids).size()

torch.Size([1, 5, 768])

In [133]:
#Adding classification head
class Trans_Enc_Classification(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.enc = Trans_Encoder(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size,config.num_labels)
        
    def forward(self,x):
        x = self.enc(x)[:,0,:]
        x = self.dropout(x)
        x = self.classifier(x)
        return x

In [139]:
config.num_labels =3
enc_clf = Trans_Enc_Classification(config)
enc_clf(input.input_ids)

tensor([[ 0.4007, -1.1916,  0.3901]], grad_fn=<AddmmBackward0>)